# Premier League Data Analysis

Notebook interactivo para el análisis de datos de la Premier League.

## 📋 Índice
1. [Configuración](#configuración)
2. [Carga de Datos](#carga-de-datos)
3. [Análisis Exploratorio](#análisis-exploratorio)
4. [Visualizaciones](#visualizaciones)
5. [Estadísticas Avanzadas](#estadísticas-avanzadas)

## 🔧 Configuración

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Importar scripts del proyecto
import sys
sys.path.append('../src')
from data_collection import PremierLeagueDataCollector
from analysis import PremierLeagueAnalyzer
from visualization import PremierLeagueVisualizer

print("✅ Librerías importadas correctamente")

## 📊 Carga de Datos

In [ ]:
# Cargar datos desde archivos CSV
try:
    # Cargar datos de la temporada 2023
    matches_df = pd.read_csv('../data/matches_2023.csv')
    standings_df = pd.read_csv('../data/standings_2023.csv')
    teams_df = pd.read_csv('../data/teams.csv')
    
    print(f"✅ Partidos cargados: {len(matches_df)} registros")
    print(f"✅ Tabla de posiciones cargada: {len(standings_df)} equipos")
    print(f"✅ Equipos cargados: {len(teams_df)} equipos")
    
except FileNotFoundError:
    print("❌ No se encontraron archivos de datos")
    print("Ejecuta primero el script de recolección de datos:")
    print("python src/data_collection.py")

## 🔍 Análisis Exploratorio

In [ ]:
# Información básica de los datos
print("=== INFORMACIÓN DE PARTIDOS ===")
matches_df.info()
print("\n=== INFORMACIÓN DE TABLA DE POSICIONES ===")
standings_df.info()

In [ ]:
# Estadísticas descriptivas
print("=== ESTADÍSTICAS DE PARTIDOS ===")
print(matches_df.describe())

print("\n=== ESTADÍSTICAS DE TABLA DE POSICIONES ===")
print(standings_df.describe())

In [ ]:
# Verificar datos nulos
print("=== DATOS NULOS EN PARTIDOS ===")
print(matches_df.isnull().sum())

print("\n=== DATOS NULOS EN TABLA DE POSICIONES ===")
print(standings_df.isnull().sum())

## 📈 Visualizaciones

In [ ]:
# 1. Distribución de goles
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Goles locales
matches_df['home_score'].hist(bins=15, ax=axes[0,0], alpha=0.7, color='skyblue')
axes[0,0].set_title('Distribución de Goles Locales')
axes[0,0].set_xlabel('Goles')
axes[0,0].set_ylabel('Frecuencia')

# Goles visitantes
matches_df['away_score'].hist(bins=15, ax=axes[0,1], alpha=0.7, color='lightcoral')
axes[0,1].set_title('Distribución de Goles Visitantes')
axes[0,1].set_xlabel('Goles')
axes[0,1].set_ylabel('Frecuencia')

# Total de goles
total_goals = matches_df['home_score'] + matches_df['away_score']
total_goals.hist(bins=15, ax=axes[1,0], alpha=0.7, color='lightgreen')
axes[1,0].set_title('Distribución de Total de Goles')
axes[1,0].set_xlabel('Total de Goles')
axes[1,0].set_ylabel('Frecuencia')

# Resultados
results = []
for _, match in matches_df.iterrows():
    if match['home_score'] > match['away_score']:
        results.append('Victoria Local')
    elif match['home_score'] < match['away_score']:
        results.append('Victoria Visitante')
    else:
        results.append('Empate')

pd.Series(results).value_counts().plot(kind='bar', ax=axes[1,1], 
                                        color=['skyblue', 'lightcoral', 'lightgray'])
axes[1,1].set_title('Resultados de Partidos')
axes[1,1].set_ylabel('Cantidad')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# 2. Tabla de posiciones
fig = go.Figure(data=[go.Table(
    header=dict(
        values=['Pos', 'Equipo', 'PJ', 'PG', 'PE', 'PP', 'PTS', 'GF', 'GC', 'DG'],
        fill_color='lightblue',
        align='left'
    ),
    cells=dict(
        values=[
            standings_df['position'],
            standings_df['team'],
            standings_df['played_games'],
            standings_df['won'],
            standings_df['draw'],
            standings_df['lost'],
            standings_df['points'],
            standings_df['goals_for'],
            standings_df['goals_against'],
            standings_df['goal_difference']
        ],
        fill_color='lavender',
        align='left'
    )
)])

fig.update_layout(title='Tabla de Posiciones Premier League 2023')
fig.show()

In [ ]:
# 3. Goles a favor vs en contra
fig = px.scatter(
    standings_df,
    x='goals_for',
    y='goals_against',
    size='points',
    color='position',
    hover_name='team',
    title='Goles a Favor vs Goles en Contra',
    labels={
        'goals_for': 'Goles a Favor',
        'goals_against': 'Goles en Contra',
        'points': 'Puntos',
        'position': 'Posición'
    }
)

# Línea de referencia (goles a favor = goles en contra)
fig.add_shape(
    type="line",
    x0=standings_df['goals_for'].min(),
    y0=standings_df['goals_for'].min(),
    x1=standings_df['goals_for'].max(),
    y1=standings_df['goals_for'].max(),
    line=dict(color="gray", dash="dash")
)

fig.show()

## 🎯 Estadísticas Avanzadas

In [ ]:
# 1. Estadísticas básicas de la temporada
total_matches = len(matches_df)
total_goals = (matches_df['home_score'] + matches_df['away_score']).sum()
avg_goals_per_match = total_goals / total_matches

# Resultados
home_wins = len(matches_df[matches_df['home_score'] > matches_df['away_score']])
away_wins = len(matches_df[matches_df['home_score'] < matches_df['away_score']])
draws = len(matches_df[matches_df['home_score'] == matches_df['away_score']])

print("=== ESTADÍSTICAS DE LA TEMPORADA ===")
print(f"Total de partidos: {total_matches}")
print(f"Total de goles: {total_goals}")
print(f"Promedio de goles por partido: {avg_goals_per_match:.2f}")
print(f"Victorias locales: {home_wins} ({home_wins/total_matches*100:.1f}%)")
print(f"Victorias visitantes: {away_wins} ({away_wins/total_matches*100:.1f}%)")
print(f"Empates: {draws} ({draws/total_matches*100:.1f}%)")

In [ ]:
# 2. Análisis por equipos
# Estadísticas como local
home_stats = matches_df.groupby('home_team').agg({
    'home_score': ['sum', 'mean', 'count'],
    'away_score': ['sum', 'mean']
}).round(2)

home_stats.columns = ['Goles_Favor_Local', 'Promedio_Goles_Local', 'Partidos_Local', 
                      'Goles_Contra_Local', 'Promedio_Goles_Contra_Local']

# Estadísticas como visitante
away_stats = matches_df.groupby('away_team').agg({
    'away_score': ['sum', 'mean', 'count'],
    'home_score': ['sum', 'mean']
}).round(2)

away_stats.columns = ['Goles_Favor_Visitante', 'Promedio_Goles_Visitante', 'Partidos_Visitante',
                       'Goles_Contra_Visitante', 'Promedio_Goles_Contra_Visitante']

print("=== ESTADÍSTICAS COMO LOCAL (Top 5) ===")
print(home_stats.head())

print("\n=== ESTADÍSTICAS COMO VISITANTE (Top 5) ===")
print(away_stats.head())

In [ ]:
# 3. Equipos más goleadores
team_goals = pd.DataFrame()

# Goles como local
home_goals = matches_df.groupby('home_team')['home_score'].sum().reset_index()
home_goals.columns = ['team', 'home_goals']

# Goles como visitante
away_goals = matches_df.groupby('away_team')['away_score'].sum().reset_index()
away_goals.columns = ['team', 'away_goals']

# Combinar
team_goals = pd.merge(home_goals, away_goals, on='team', how='outer').fillna(0)
team_goals['total_goals'] = team_goals['home_goals'] + team_goals['away_goals']
team_goals = team_goals.sort_values('total_goals', ascending=False)

print("=== EQUIPOS MÁS GOLEADORES ===")
print(team_goals.head(10))

In [ ]:
# 4. Visualización de equipos más goleadores
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Top 10 Equipos Más Goleadores', 'Distribución de Goles'),
    specs=[[{"type": "bar"}, {"type": "pie"}]]
)

# Top 10 equipos más goleadores
top_10 = team_goals.head(10)
fig.add_trace(
    go.Bar(x=top_10['team'], y=top_10['total_goals'], 
           name='Total de Goles', marker_color='lightblue'),
    row=1, col=1
)

# Distribución de goles (top 5)
top_5 = team_goals.head(5)
fig.add_trace(
    go.Pie(labels=top_5['team'], values=top_5['total_goals'], 
           name='Distribución de Goles'),
    row=1, col=2
)

fig.update_layout(
    title='Análisis de Goles por Equipo',
    height=500,
    showlegend=False
)

fig.update_xaxes(tickangle=45, row=1, col=1)
fig.show()

## 🚀 Próximos Pasos

### Análisis Predictivo
- Crear modelo de predicción de resultados
- Analizar factores que influyen en victorias
- Sistema de recomendación de partidos

### Análisis Temporal
- Tendencias a lo largo de la temporada
- Análisis de rachas de equipos
- Comparación entre temporadas

### Machine Learning
- Clustering de equipos por estilo de juego
- Modelo de clasificación de resultados
- Análisis de componentes principales

In [ ]:
# Guardar análisis preliminar
analysis_summary = {
    'total_matches': len(matches_df),
    'total_goals': total_goals,
    'avg_goals_per_match': avg_goals_per_match,
    'home_win_rate': home_wins/total_matches*100,
    'away_win_rate': away_wins/total_matches*100,
    'draw_rate': draws/total_matches*100,
    'top_scorer_team': team_goals.iloc[0]['team'],
    'top_scorer_goals': team_goals.iloc[0]['total_goals']
}

print("\n=== RESUMEN DEL ANÁLISIS ===")
for key, value in analysis_summary.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

print("\n✅ Análisis completado exitosamente!")